# Tahoe Forest Health Threshold Analysis

## Setup

##### Links
* https://caregionalresourcekits.org//sierra.html#forest_res

In [ ]:
# import packages
from utils import *
import os
import arcpy
from arcpy.sa import *
# from arcpy.ia import *
# from arcpy.ddd import *
import pandas as pd
from pathlib import Path

# setup workspace variables
arcpy.env.workspace = r"F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\ForestHealth_ThresholdUpdate.gdb"
arcpy.env.overwriteOutput = True

#format paths
sdeBase   = Path("F:\GIS\DB_CONNECT\Vector.sde")
# workspace = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb")
downloads = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\SNV_RRK")
output    = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct")

# base layers
trpa_boundary       = sdeBase / "SDE.Jurisdictions\SDE.TRPA_bdy"
mgmt_areas          = sdeBase / "SDE.Jurisdictions\SDE.ForestManagementZone_USFS"

# raw raster data downloaded from SNVRRK
veg_type            = downloads / "CWHR_vegetation\RRK_Fveg_WHRtype_2023Apr_4regions_v2.tif"
stand_density       = downloads / "TPA_2021_30m\TPA_2021_30m.tif"
basal_area          = downloads / "BASATOT_2021_30m\BASATOT_2021_30m.tif"
seral_stage         = downloads / "SeralStage_EML_2021\SeralStage_EML_2021.tif"
canopy_cover        = downloads / "CFO_CanopyCover2020Summer\CFO_CanopyCover2020Summer.tif"
fire_prob_low       = downloads / "ProbabilityLowFireSev_202308\ProbabilityLowFireSev_202308.tif"
fire_prob_moderate  = downloads / "ProbabilityModerateFireSev_202308\ProbabilityModerateFireSev_202308.tif"
fire_prob_high      = downloads / "ProbabilityHighFireSev_202308\ProbabilityHighFireSev_202308.tif"

# === Forest Type Mapping ===
veg_map = {
    "JPN": 1, "EPN": 1, "JUN": 1, "LPN": 1,   # Jeffrey Pine
    "SMC": 2, "WFR": 2,                       # White fir-mixed conifer
    "RFR": 3,                                 # Red fir
    "SCN": 4,                                 # Sub Alpine Conifer
    "ASP": 5                                  # Aspen
}
# The forest type mapping is based on the CWHR vegetation types.
def build_forest_type_raster(input_raster):
    remap = RemapValue([[k, v] for k, v in veg_map.items()])
    return Reclassify(input_raster, "WHRTYPE", remap, "NODATA")

In [ ]:
# list datasets in workspace
feature_data = arcpy.ListFeatureClasses()
raster_data  = arcpy.ListRasters()
table_data   = arcpy.ListTables()
# print feature classes, then rasters, then tables
print("Feature Classes:")
for fc in feature_data:
    print(fc)
print("\nRasters:")
for raster in raster_data:
    print(raster)
print("\nTables:")
for table in table_data:
    print(table)

### Extract to Tahoe

In [ ]:
# save a verions of each input tif at full extent and extract the extent to Tahoe
# create a list of the rasters to be clipped
raster_list = [
    veg_type,
    stand_density,
    basal_area,
    seral_stage,
    canopy_cover,
    fire_prob_low,
    fire_prob_moderate,
    fire_prob_high
]
# names of the rasters to be clipped
raster_names = [
    "veg_type_sierra",
    "stand_density_sierra",
    "basal_area_sierra",
    "seral_stage_sierra",
    "canopy_cover_sierra",
    "fire_prob_low_sierra",
    "fire_prob_moderate_sierra",
    "fire_prob_high_sierra"
]

# save a version of each input tif at full extent
for raster, name in zip(raster_list, raster_names):
    # save a version of each input tif at full extent
    out_raster = f"{name}"
    arcpy.sa.Raster(str(raster)).save(out_raster)
    print(f"Saved {name} to full extent")
# extract by mask and add _tahoe to the name
for name in raster_names:
    # extract by mask
    # format output name to take off _sieera and add _tahoe
    output_name = name.replace("_sierra", "_tahoe")
    out_extract = extract_by_mask_to_tahoe_extent(name, output_name)
    print(f"Extracted {output_name} to Tahoe")

## Stand Density

This code defines a function, `assess_density_targets`, that evaluates whether each pixel in a set of forest raster datasets meets specific stand density and basal area thresholds, based on forest type and seral stage. The function uses a dictionary called `thresholds` to store the minimum required values for stand density (stems per acre) and basal area (square feet per acre) for each combination of forest type and seral stage. For example, Jeffrey Pine in the early seral stage must have less than 200 stems per acre and 30 square feet of basal area.

The function initializes an output raster where all pixels are set to -1, indicating "not classified." It then iterates through each forest type and seral stage combination in the thresholds dictionary. For each combination, it uses logical conditions to identify pixels that both match the forest type and seral stage and meet or exceed the required thresholds ("meets"). These pixels are assigned a value of 1 in the output raster. Pixels that match the forest type and seral stage but fail to meet at least one threshold ("fails") are assigned a value of 0. The function uses the `Con` tool from ArcPy to perform these conditional assignments efficiently across the raster.

After defining the function, the code loads four rasters: forest type, seral stage, stand density, and basal area. The forest type raster is created by reclassifying the vegetation type raster using a separate function, `build_forest_type_raster`. The assessment function is then called to generate a new raster, where each pixel is classified as 1 (meets both targets), 0 (fails at least one target), or -1 (not classified). This workflow allows for spatially explicit evaluation of forest stand conditions against threshold targets.

In [ ]:
# === Stand Density Threshold Assessment ===
# Function to evaluate pixels against threshold targets
def assess_density_targets(forest_type_raster, seral_stage_raster, stand_density_raster, basal_area_raster):
    thresholds = {
        (1, 1): (200, 30),  # JP Early
        (1, 2): (70, 80),   # JP Mid
        (1, 3): (60, 100),  # JP Late
        (2, 1): (300, 40),  # WF Early
        (2, 2): (100, 150), # WF Mid
        (2, 3): (80, 200),  # WF Late
        (3, 1): (300, 50),  # RF Early
        (3, 2): (100, 250), # RF Mid
        (3, 3): (80, 350),  # RF Late
        (4, 1): (400, 60),  # Sub Alpine Conifer Early
        (4, 2): (150, 300), # Sub Alpine Conifer Mid
        (4, 3): (120, 450), # Sub Alpine Conifer Late
        (5, 1): (400, 60),  # Aspen Early
        (5, 2): (150, 300), # Aspen Mid
        (5, 3): (120, 450), # Aspen Late
    }
    # Initialize output raster with -1 (not classified)
    out_raster = Int(-1)  # default: not classified

    for (ftype, stage), (min_stems, min_ba) in thresholds.items():
        # Meets or exceeds both targets
        meets = (
            (forest_type_raster == ftype) &
            (seral_stage_raster == stage) &
            (stand_density_raster >= min_stems) &
            (basal_area_raster >= min_ba)
        )
        # Forest type/seral match, but fails to meet at least one target
        fails = (
            (forest_type_raster == ftype) &
            (seral_stage_raster == stage) &
            ~meets
        )
        # Forest type/seral classification of stand density
        out_raster = Con(meets, 1, out_raster)
        out_raster = Con(fails, 0, out_raster)
        # classification of non-forest types
        print("Forest Type: ", ftype, "Seral Stage: ", stage, "Min Stems: ", min_stems, "Min BA: ", min_ba)
    # set nodata to -1
    out_raster = SetNull(out_raster == -1, out_raster)
    # Check if the output raster already exists and delete it
    output_path = "stand_density_threshold_assessment"
    if arcpy.Exists(output_path):
        arcpy.management.Delete(output_path)
    # Save the raster
    out_raster.save(output_path)
    print(f"Saved: {output_path}")
    # Build a raster attribute table
    arcpy.BuildRasterAttributeTable_management(out_raster, "Overwrite")
    # Add acres to the raster attribute table
    add_acres(out_raster)
    # within atainment / out of attainment field 1= attainment, 0 = not attainment
    arcpy.AddField_management(out_raster, "Attainment", "SHORT")
    arcpy.CalculateField_management(out_raster, "Attainment", "1", "PYTHON3")
    # set attainment to 0 where the raster value is 0
    arcpy.CalculateField_management(out_raster, "Attainment", "0", "PYTHON3", where_clause="VALUE = 0")
    # Save again to ensure all updates are written
    out_raster.save(output_path)
    print(f"Saved w/ acres-attainment: {output_path}")

# Load rasters
veg_type_r      = Raster("veg_type_tahoe")
seral_stage_r   = Raster("seral_stage_tahoe")
stand_density_r = Raster("stand_density_tahoe")
basal_area_r    = Raster("basal_area_tahoe")

# Build classification and run assessment
forest_type_r = build_forest_type_raster(veg_type_r)
assessment_r  = assess_density_targets(forest_type_r, seral_stage_r, stand_density_r, basal_area_r)

## Seral Stage

This code performs a classification and tabulation of forest seral stages by combining information from three raster datasets: forest type, seral stage, and canopy cover. The main function, `classify_seral_stage`, first prepares the canopy cover raster by filling any null values with zero. It then creates a binary canopy class: for most forest types, pixels with canopy cover ≥50% are considered "closed canopy," but for Jeffrey Pine (forest type code 1), the threshold is lowered to 40%. This logic ensures that the canopy classification is tailored to the ecological characteristics of each forest type.

Next, the function combines the seral stage and canopy class rasters into a single raster (`combined_class_r`) that encodes five possible seral stage classes: early, mid (closed canopy), mid (open canopy), late (open canopy), and late (closed canopy). This is achieved using nested `Con` statements, which assign a unique class code based on the combination of seral stage and canopy closure.

The `TabulateArea` tool is then used to summarize the area of each seral stage class within each forest type, producing a table where each row represents a forest type and each column represents a seral stage class. The function reads this table, calculates the area in both square meters and acres, and computes the percentage of each seral stage class within each forest type. It then compares these percentages to user-defined "desired conditions" (target percentage ranges for each class and forest type) and classifies each result as "on target," "below target," or "above target."

Finally, the results are organized into a pandas DataFrame with descriptive labels for forest type and seral stage, and the DataFrame is saved as a CSV file. This workflow provides a clear, quantitative assessment of how current forest conditions compare to threshold targets.

In [ ]:
# === Seral Stage Classification ===
# Function to classify seral stages based on forest type and canopy cover
# and tabulate the results
def classify_seral_stage(forest_type_r, seral_stage_r, canopy_cover_r, desired_conditions, output_table="seral_tabulation"):
    cell_size = forest_type_r.meanCellWidth
    m2_to_acres = 1 / 4046.8564224

    canopy_cover_r = Con(IsNull(canopy_cover_r), 0, canopy_cover_r)
    canopy_class_default = Con(canopy_cover_r >= 50, 1, 0)
    canopy_class_jp = Con((forest_type_r == 1) & (canopy_cover_r >= 40), 1, 0)
    canopy_class_r = Con(forest_type_r == 1, canopy_class_jp, canopy_class_default)
    canopy_class_r = Con(IsNull(canopy_class_r), 0, canopy_class_r)

    combined_class_r = Con(seral_stage_r == 1, 1,
                     Con((seral_stage_r == 2) & (canopy_class_r == 0), 2,
                     Con((seral_stage_r == 2) & (canopy_class_r == 1), 3,
                     Con((seral_stage_r == 3) & (canopy_class_r == 1), 4,
                     Con((seral_stage_r == 3) & (canopy_class_r == 0), 5, None)))))

    TabulateArea(
        in_zone_data=forest_type_r,
        zone_field="Value",
        in_class_data=combined_class_r,
        class_field="Value",
        out_table=output_table,
        processing_cell_size=cell_size
    )

    if not arcpy.Exists(output_table):
        raise FileNotFoundError(f"Output table does not exist: {output_table}")

    fields = [f.name for f in arcpy.ListFields(output_table) if f.name.startswith("VALUE_")]
    if not fields:
        raise ValueError("No VALUE_* fields found in tabulation output.")

    raw_counts = {}
    with arcpy.da.SearchCursor(output_table, ["Value"] + fields) as cursor:
        for row in cursor:
            forest_code = row[0]
            counts = list(row[1:])
            raw_counts[forest_code] = counts

    for code in desired_conditions:
        if code not in raw_counts:
            raw_counts[code] = [0] * len(fields)

    data = []
    for forest_code, targets in desired_conditions.items():
        class_counts = raw_counts[forest_code]
        total = sum(class_counts)
        for i, field in enumerate(fields):
            seral_code = int(field.replace("VALUE_", ""))
            count = class_counts[i]
            area_m2 = count
            area_acres = area_m2 * m2_to_acres
            percent = round((count / total) * 100, 2) if total > 0 else 0
            desired_range = targets.get(seral_code, (0, 0))
            status = (
                'on target' if desired_range[0] <= percent <= desired_range[1]
                else 'below target' if percent < desired_range[0]
                else 'above target'
            )
            data.append({
                "Forest Type Code": forest_code,
                "Seral Stage Code": seral_code,
                "Desired % Range": f"{desired_range[0]}–{desired_range[1]}%",
                "Pixel Count": count,
                "Area (m²)": round(area_m2, 0),
                "Area (acres)": round(area_acres, 0),
                "Current Area %": percent,
                "Classification": status
            })

    seral_stage_map = {
        1: "Early",
        2: "Mid (closed canopy)",
        3: "Mid (open canopy)",
        4: "Late (open canopy)",
        5: "Late (closed canopy)"
    }
    veg_type_map = {
        1: "Jeffrey Pine",
        2: "White Fir",
        3: "Red Fir",
        4: "Sub Alpine Conifer",
        5: "Aspen"
    }

    df = pd.DataFrame(data)
    df["Seral Stage"] = df["Seral Stage Code"].map(seral_stage_map)
    df["Forest Type"] = df["Forest Type Code"].map(veg_type_map)
    df = df[["Forest Type", "Seral Stage", "Desired % Range", "Classification", "Current Area %", "Area (m²)", "Area (acres)"]]

    return df

# Desired ranges
desired_conditions = {
    1: {1: (5, 15),   2: (5, 10), 3: (25, 30), 4: (45, 50), 5: (5, 10)},    # Jeffrey Pine
    2: {1: (10, 20),  2: (5, 15), 3: (10, 15), 4: (30, 40), 5: (20, 30)},   # White Fir
    3: {1: (10, 20),  2: (20, 30), 3: (5, 15), 4: (15, 25), 5: (25, 35)},   # Red Fir
    4: {1: (10, 20),  2: (10, 20), 3: (5, 15), 4: (15, 25), 5: (10, 20)},   # Sub Alpine Conifer
    5: {1: (10, 20),  2: (5, 15), 3: (10, 20), 4: (15, 25), 5: (25, 35)}    # Aspen
}

# Load and reclassify input rasters
veg_type_r_raw = Raster("veg_type_tahoe")
forest_type_r  = build_forest_type_raster(veg_type_r_raw)
seral_stage_r  = Raster("seral_stage_tahoe")
canopy_cover_r = Raster("canopy_cover_tahoe")

# Run classification
df = classify_seral_stage(forest_type_r, seral_stage_r, canopy_cover_r, desired_conditions)

# Save results
df.to_csv(output / "seral_stage_with_classification.csv", index=False)
print("Saved: seral_stage_with_classification.csv")

## Functional Fire

This code provides a workflow for classifying and analyzing fire severity across a landscape using raster data. The first part defines a function, `dominant_fire_severity`, which determines the dominant fire severity class for each pixel by comparing three input rasters: high, moderate, and low fire probability. It uses the `Con` function to assign a value of 1 if the low severity probability is highest, 2 if moderate is highest, and 3 otherwise (implying high severity dominates). The result is converted to an integer raster and saved to disk. The function also builds an attribute table for the output raster and adds an "Acres" field, which is calculated for each class, then saves the updated raster again.

The second function, `identify_fire_severity`, overlays the dominant fire severity raster with management zones. It first converts the raster to polygons, adds and calculates an "Acres" field for each polygon, and then performs an identity analysis to spatially join the polygons with management zone features. The result is a new feature class that contains both the dominant fire severity class and the management zone information, with acreage recalculated for the output. This workflow enables spatial analysis of fire severity patterns in relation to management areas, supporting landscape-level fire and resource planning.

In [ ]:
# === Fire Severity Classification ===
# function to classify fire severity based on the downloaded rasters
def dominant_fire_severity(highsev, modsev, lowsev):
    # Calculate dominant class
    # make rasters
    highsev = Raster(highsev)
    modsev  = Raster(modsev)
    lowsev  = Raster(lowsev)
    # Use Con and logical operators to determine max
    dominant_class = Con(
        (lowsev >= modsev) & (lowsev >= highsev), 1,
        Con((modsev >= lowsev) & (modsev >= highsev), 2, 3)
    )
    # Convert to integer raster
    dominant_class_int = Raster(Int(dominant_class))
    # Save the raster
    dominant_class_int.save("functionalfire_dominant_severity_tahoe")
    # Build an attribute table
    arcpy.BuildRasterAttributeTable_management(dominant_class_int, "Overwrite")
    # Calculate Acres field
    add_acres(dominant_class_int)
    # Save the updated raster
    dominant_class_int.save("functionalfire_dominant_severity_tahoe")
    print("Updated raster: functionalfire_dominant_severity_tahoe")

# function to overlay the fire serverity dominant class with the management zones
def identify_fire_severity(dominant_class_raster, mgmt_areas, 
                           polygon = "functionalfire_dominant_severity_tahoe_polygon", 
                           output_name = "identity_fire_severity_mgmt_zones"):
    # Identify the dominant fire severity class with management zones
    # export to feature class
    arcpy.RasterToPolygon_conversion(
        in_raster= dominant_class_raster,
        out_polygon_features=polygon,
        simplify="NO_SIMPLIFY",
        raster_field="Value"
    )
    # add acres field
    arcpy.management.AddField(polygon, "Acres", "DOUBLE")
    # calculate acres field
    arcpy.management.CalculateField(polygon, "Acres", "!shape.area@acres!", "PYTHON3")
    # identiy with management zones
    arcpy.analysis.Identity(
        in_features=polygon,
        identity_features=str(mgmt_areas),
        out_feature_class=output_name
    )
    # calculate acres
    arcpy.management.CalculateField(output_name, "Acres", "!shape.area@acres!", "PYTHON3")
    print(f"Exported: {output_name}")

# Load the raw fire severity rasters
highsev_tahoe = "fire_prob_high_tahoe"
modsev_tahoe  = "fire_prob_moderate_tahoe"
lowsev_tahoe  = "fire_prob_low_tahoe"

# Run the function to classify fire severity
dominant_fire_severity(highsev_tahoe, modsev_tahoe, lowsev_tahoe)
# Run the function to identify fire severity with management zones
identify_fire_severity("functionalfire_dominant_severity_tahoe", mgmt_areas, polygon = "functionalfire_dominant_severity_tahoe_polygon", output_name = "identity_fire_severity_mgmt_zones")